In [ ]:
import sys
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, minmax_scale
import tensorflow as tf
from tensorflow.keras.losses import mean_absolute_error, binary_crossentropy
import numpy as np

from holodecml.data import load_scaled_datasets, load_raw_datasets, calc_z_bins
from holodecml.models import attention_net_loss

In [ ]:
path_data = "/glade/p/cisl/aiml/ai4ess_hackathon/holodec/"
num_particles = "large"
output_cols = ["x", "y", "z", "d", "hid"]
scaler_out = MinMaxScaler()
subset = 0.25
num_z_bins = False
mass = False


In [ ]:
train_inputs,\
train_outputs,\
valid_inputs,\
valid_outputs = load_scaled_datasets(path_data,
                                     num_particles,
                                     output_cols,
                                     scaler_out,
                                     subset,
                                     num_z_bins,
                                     mass)


In [ ]:
train_inputs_raw, train_outputs_raw = load_raw_datasets(path_data,
                                                        num_particles, 
                                                        "train",
                                                        output_cols,
                                                        subset)


In [ ]:
def make_template(df, num_images):
    max_particles = df['hid'].value_counts().max()
    size = (max_particles * num_images, 1) 
    x = np.random.uniform(low=df['x'].min(), high=df['x'].max(), size=size)
    y = np.random.uniform(low=df['y'].min(), high=df['y'].max(), size=size)
    z = np.random.uniform(low=df['z'].min(), high=df['z'].max(), size=size)
    d = np.random.uniform(low=df['d'].min(), high=df['d'].max(), size=size)
    prob = np.zeros(d.shape)
    template = np.hstack((x, y ,z ,d ,prob))
    template = template.reshape((num_images, max_particles, -1))
    return template

In [ ]:
template = make_template(train_outputs_raw, train_inputs_raw.shape[0])
template[:,:50,-1] = 1

In [ ]:
y_true = tf.Variable(template)


In [ ]:
y_true.shape

In [ ]:
def attention_net_loss(y_true, y_pred):
    # y_true and y_pred will have shape (batch_size x max_num_particles x 5)
    loss_real = tf.reduce_mean(tf.abs(y_true[y_true[:, :, -1] > 0] - y_pred[y_true[:, :, -1] > 0]))
    
    print(loss_real)
    loss_bce = binary_crossentropy(tf.reshape(y_true[:,:,-1], (y_true.shape[0]*y_true.shape[1])),
                                   tf.reshape(y_pred[:,:,-1], (y_true.shape[0]*y_true.shape[1])))
    print(loss_bce)
    loss_total = loss_real + loss_bce
    return loss_total

In [ ]:
loss_real = attention_net_loss(y_true, y_true)
print(loss_real)


In [2]:
!python train_attn.py ../../config/attn.yml

Starting script...
Loading data...
2020-09-30 14:24:02.198468: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-30 14:24:02.238384: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:61:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2020-09-30 14:24:02.252296: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-30 14:24:02.262676: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-09-30 14:24:02.272090: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2020-09-30 14:24:02.274373: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully 